In [30]:
# Copyright 2025 Luke Moffett
# Licensed under the Apache License, Version 2.0

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tqdm
import torch

from IPython.display import display
from clz_or_cls import analysis, hot

tqdm.tqdm.pandas()
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

plt.rcParams['figure.dpi'] = 100
%matplotlib inline

#### Load Original Dataset

In [ ]:
hot_df = hot.hot_metadata_df()
hot_df.head(3)

In [ ]:
csvs = analysis.get_hot_classification_csvs()
csvs

In [33]:
result_df = analysis.load_and_concatenate_csvs(csvs, levels=['model', 'size', 'xshot', 'ratio', 'train', 'test', 'class'])

result_df['ID'] = result_df['ID'].astype(int)
result_df['ratio'] = result_df['ratio'].astype(float)

result_df.head()

,idx,ID,score,explanation,batch,model,size,xshot,ratio,train,test,class
0,0,0,0.2,\nThe main factors in determining whether or n...,0,llama2,7B,0-shot,0.5,0shot,phonetic,offensive
1,1,1,0.6,\nThe main factors in determining whether or n...,0,llama2,7B,0-shot,0.5,0shot,phonetic,offensive
2,2,2,0.6,\nThe main factors in determining whether or n...,0,llama2,7B,0-shot,0.5,0shot,phonetic,offensive
3,3,3,0.2,\nThe main factors in determining whether or n...,0,llama2,7B,0-shot,0.5,0shot,phonetic,offensive
4,4,4,0.6,\nThe main factors in determining whether or n...,0,llama2,7B,0-shot,0.5,0shot,phonetic,offensive


In [34]:
orig_cols = result_df.columns
idx_cols = ['model', 'size', 'xshot', 'train', 'test', 'ID', 'idx', 'ratio']

keep_cols = [col for col in orig_cols if col not in idx_cols]

all_class_df = result_df[result_df['class'] == 'hateful']
all_class_df = all_class_df.set_index(idx_cols)
all_class_df.columns = pd.MultiIndex.from_tuples(
   [('hateful', col) for col in keep_cols]
)

for clazz in ['offensive', 'toxic']:
    class_results = result_df[result_df['class'] == clazz]
    class_results = class_results.set_index(idx_cols)
    class_results.columns = pd.MultiIndex.from_tuples([(clazz, col) for col in keep_cols])
    all_class_df = all_class_df.merge(class_results, left_index=True, right_index=True, how='outer')

all_class_df.columns = ["_".join(a) for a in all_class_df.columns.to_flat_index()]
all_class_df.index = all_class_df.index.droplevel('xshot').droplevel('train').droplevel('idx')
all_class_df

hateful_score  \
model   size        test     ID   ratio                  
llama2  7B          phonetic 0    0.5              0.2   
                             1    0.5              0.6   
                             2    0.5              0.6   
                             3    0.5              0.2   
                             4    0.5              0.5   
...                                                ...   
mistral 7B-shielded visual   3476 0.5              0.1   
                             3477 0.5              0.3   
                             3478 0.5              0.1   
                             3479 0.5              0.1   
                             3480 0.5              0.1   

                                                                       hateful_explanation  \
model   size        test     ID   ratio                                                      
llama2  7B          phonetic 0    0.5    \nThe main factors in determining whether or n...   
                             1    0.5    \nThe main factors in determining whether or n...   
                             2    0.5    \nThe main factors in determining whether or n...   
                             3    0.5    \nThe main factors in determining whether or n...   
                             4    0.5    \nThe main factors in determining whether or n...   
...                                                                                    ...   
mistral 7B-shielded visual   3476 0.5    \nThe main factors in determining whether or n...   
                             3477 0.5    \nThe main factors in determining whether or n...   
                             3478 0.5    \nThe main factors in determining whether or n...   
                             3479 0.5    \nThe main factors in determining whether or n...   
                             3480 0.5    \nThe main factors in determining whether or n...   

                                         hateful_batch hateful_class  \
model   size        test     ID   ratio                                
llama2  7B          phonetic 0    0.5                0       hateful   
                             1    0.5                0       hateful   
                             2    0.5                0       hateful   
                             3    0.5                0       hateful   
                             4    0.5                0       hateful   
...                                                ...           ...   
mistral 7B-shielded visual   3476 0.5              141       hateful   
                             3477 0.5              141       hateful   
                             3478 0.5              141       hateful   
                             3479 0.5              141       hateful   
                             3480 0.5              141       hateful   

                                         offensive_score  \
model   size        test     ID   ratio                    
llama2  7B          phonetic 0    0.5                0.2   
                             1    0.5                0.6   
                             2    0.5                0.6   
                             3    0.5                0.2   
                             4    0.5                0.6   
...                                                  ...   
mistral 7B-shielded visual   3476 0.5                0.1   
                             3477 0.5                0.7   
                             3478 0.5                0.1   
                             3479 0.5                0.1   
                             3480 0.5                0.2   

                                                                     offensive_explanation  \
model   size        test     ID   ratio                                                      
llama2  7B          phonetic 0    0.5    \nThe main factors in determining whether or n...   
                             1    0.5    \nThe main factors in 

In [ ]:
labeled_df = hot_df.merge(all_class_df.reset_index(), on='ID')
labeled_df = labeled_df.set_index(['model', 'size', 'test', 'ratio', 'ID']).drop('Unnamed: 0_x', axis=1)
labeled_df.head(3)

In [36]:
from sklearn.metrics import accuracy_score
def find_optimal_threshold(true_labels, predicted_scores):
    best_threshold = 0.0
    best_accuracy = 0.0

    for threshold in np.linspace(0, 1, 101):
        # Convert probabilities to binary predictions based on the current threshold
        predicted_labels = (predicted_scores >= threshold).astype(int)

        # Calculate accuracy
        accuracy = accuracy_score(true_labels, predicted_labels)

        # Check if this threshold gives a better accuracy
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_threshold = threshold

    return best_threshold, best_accuracy

thresholds = {}
for model_name in labeled_df.index.get_level_values('model').unique():
    model = labeled_df.loc[model_name, :, 'clean']
    thresholds[model_name] = {}
    for clz in ['hateful', 'offensive', 'toxic']:
        optimal_threshold, optimal_accuracy = find_optimal_threshold(model[f'{clz}_hard'], model[f'{clz}_score'])
        print(f"{clz} Optimal Threshold: {optimal_threshold}, Accuracy: {optimal_accuracy}")
        thresholds[model_name][clz] = optimal_threshold

thresholds

hateful Optimal Threshold: 0.91, Accuracy: 0.8045936395759717
offensive Optimal Threshold: 0.41000000000000003, Accuracy: 0.7724381625441696
toxic Optimal Threshold: 0.7000000000000001, Accuracy: 0.8522968197879859
hateful Optimal Threshold: 0.81, Accuracy: 0.8134275618374558
offensive Optimal Threshold: 0.7000000000000001, Accuracy: 0.757243816254417
toxic Optimal Threshold: 0.91, Accuracy: 0.8212014134275618


{'llama2': {'hateful': 0.91,
  'offensive': 0.41000000000000003,
  'toxic': 0.7000000000000001},
 'mistral': {'hateful': 0.81, 'offensive': 0.7000000000000001, 'toxic': 0.91}}

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, auc, roc_auc_score

def metric_package(model, size, test, ratio, column):
    df = labeled_df.loc[model, size, test, ratio]
    labels_all = df[f'{column}_hard']
    predictions_all = df[f'{column}_thresh']
    predictions_soft = df[f'{column}_score']

    if 'shielded' in size:
        clean_size = size.split('-')[0]
    else:
        clean_size = size

    def safe_auc(true, score):
        try:
            return round(roc_auc_score(true, score), 2)
        except ValueError:
            return np.nan

    def safe_count(field, value, norm=False):
        try:
            return field.value_counts(normalize=norm)[value]
        except KeyError:
            return 0
        
    if test != 'clean':
        clean_df = labeled_df.loc[model, clean_size, 'clean', 0.0]
        pairwise_delta = (df[f'{column}_score'].astype(float) - clean_df[f'{column}_score'].astype(float)).abs().mean()
    else:
        pairwise_delta = np.nan

    return (column,
            'Yes',
            safe_count(labels_all, True),\
            safe_count(labels_all, True, norm=True),\
            safe_count(predictions_all, True),\
            safe_count(predictions_all, True, norm=True),\
            predictions_soft.mean(),\
            predictions_soft.median(),\
            pairwise_delta,\
            round(precision_score(labels_all, predictions_all), 2),\
            round(recall_score(labels_all, predictions_all), 2),\
            round(f1_score(labels_all, predictions_all), 2),\
            round(accuracy_score(labels_all, predictions_all), 2),\
            safe_auc(labels_all, predictions_soft),\
            confusion_matrix(predictions_all, labels_all)),\
            (column,
            'No',
            safe_count(labels_all, False),\
            safe_count(labels_all, False, norm=True),\
            safe_count(predictions_all, False),\
            safe_count(predictions_all, False, norm=True),\
            predictions_soft.mean(),\
            predictions_soft.median(),\
            pairwise_delta,\
            round(precision_score(~labels_all, ~predictions_all), 2),\
            round(recall_score(~labels_all, ~predictions_all), 2),\
            round(f1_score(~labels_all, ~predictions_all), 2),\
            round(accuracy_score(labels_all, predictions_all), 2),\
            safe_auc(labels_all, predictions_soft),\
            confusion_matrix(predictions_all, labels_all))

from itertools import chain
def metric_table(model, size, test, ratio, thresholds={'hateful': .7, 'offensive': .7, 'toxic':.8}):

    labeled_df['hateful_thresh'] = labeled_df['hateful_score'] >= thresholds['hateful']
    labeled_df['offensive_thresh'] = labeled_df['offensive_score'] >= thresholds['offensive']
    labeled_df['toxic_thresh'] = labeled_df['toxic_score'] >= thresholds['toxic']

    categories = ['hateful', 'offensive', 'toxic']
    metrics = list(chain(*[metric_package(model, size, test, ratio, hot) for hot in categories]))
    return pd.DataFrame(metrics, columns=['Category', 'Class', 'Support', 'Support %', 'Prediction', 'Prediction %', 'Mean Score', 'Med Score', '∆ Score', 'Precision', 'Recall', 'F1-Score', 'Accuracy', 'AUC', 'Confusion']).set_index(['Category', 'Class'])

replication_metrics = metric_table('mistral', '7B', 'visual', .125, thresholds=thresholds['mistral'])
replication_metrics

In [38]:
replication_metrics[['Support', 'Precision', 'Recall', 'F1-Score', 'Accuracy']]

Support  Precision  Recall  F1-Score  Accuracy
Category  Class                                                
hateful   Yes        553       0.50    0.24      0.32      0.80
          No        2277       0.84    0.94      0.89      0.80
offensive Yes        878       0.45    0.69      0.54      0.64
          No        1952       0.82    0.62      0.70      0.64
toxic     Yes        507       0.00    0.00      0.00      0.82
          No        2323       0.82    1.00      0.90      0.82

In [39]:
hot_table_source_paper_df = pd.DataFrame({
    'Category': ['hateful', 'hateful', 'offensive', 'offensive', 'toxic', 'toxic'],
    'Class': ['yes', 'no', 'yes', 'no', 'yes', 'no'],
    'Support': [404, 3077, 862, 2616, 801, 2675],
    'Precision': [0.45, 0.92, 0.67, 0.86, 0.47, 0.94],
    'Recall': [0.34, 0.79, 0.55, 0.91, 0.86, 0.71],
    'F1-score': [0.39, 0.93, 0.61, 0.89, 0.61, 0.81],
    'Accuracy': [0.87, 0.87, 0.82, 0.82, 0.75, 0.75]
}).set_index(['Category', 'Class'])
hot_table_source_paper_df

Support  Precision  Recall  F1-score  Accuracy
Category  Class                                                
hateful   yes        404       0.45    0.34      0.39      0.87
          no        3077       0.92    0.79      0.93      0.87
offensive yes        862       0.67    0.55      0.61      0.82
          no        2616       0.86    0.91      0.89      0.82
toxic     yes        801       0.47    0.86      0.61      0.75
          no        2675       0.94    0.71      0.81      0.75

# Results

In [40]:
labeled_df.reset_index().groupby(['model', 'size', 'test', 'ratio']).count()[['ID']]

ID
model   size        test     ratio      
llama2  13B         clean    0.000  2830
                    phonetic 0.125  2830
                             0.250  2830
                             0.500  2830
                    typo     0.125  2830
                             0.250  2830
                             0.500  2830
                    visual   0.125  2830
                             0.250  2830
                             0.500  2830
        7B          clean    0.000  2830
                    phonetic 0.125  2830
                             0.250  2830
                             0.500  2830
                    typo     0.125  2830
                             0.250  2830
                             0.500  2830
                    visual   0.125  2830
                             0.250  2830
                             0.500  2830
mistral 7B          clean    0.000  2830
                    phonetic 0.125  2830
                             0.250  2830
                             0.500  2830
                    typo     0.125  2830
                             0.250  2830
                             0.500  2830
                    visual   0.125  2830
                             0.250  2830
                             0.500  2830
        7B-shielded visual   0.500  2810

In [ ]:
# Assuming df is your dataframe
# Group by 'Dataset', 'Prompting', and 'Ratio' and iterate over the groups
keys = []
metric_dfs = []
for (model, size, test, ratio), group in labeled_df.groupby(level=['model', 'size', 'test', 'ratio']):
    keys.append((model, size, test, float(ratio)))
    metrics = metric_table(model, size, test, ratio, thresholds=thresholds[model])
    metric_dfs.append(metrics)

total_metric_table = pd.concat(metric_dfs, keys=keys, names=['model', 'size', 'test', 'ratio'])
total_metric_table

In [42]:
total_metric_table.loc['mistral', :, :, 0.0]

Support  Support %  Prediction  Prediction %  \
size test  Category  Class                                                 
7B   clean hateful   Yes        553   0.195406         133      0.046996   
                     No        2277   0.804594        2697      0.953004   
           offensive Yes        878   0.310247         709      0.250530   
                     No        1952   0.689753        2121      0.749470   
           toxic     Yes        507   0.179152           1      0.000353   
                     No        2323   0.820848        2829      0.999647   

                            Mean Score  Med Score  ∆ Score  Precision  Recall  \
size test  Category  Class                                                      
7B   clean hateful   Yes      0.361519        0.3      NaN       0.59    0.14   
                     No       0.361519        0.3      NaN       0.82    0.98   
           offensive Yes      0.444170        0.4      NaN       0.63    0.51   
                     No       0.444170        0.4      NaN       0.80    0.87   
           toxic     Yes      0.509435        0.6      NaN       1.00    0.00   
                     No       0.509435        0.6      NaN       0.82    1.00   

                            F1-Score  Accuracy   AUC  \
size test  Category  Class                             
7B   clean hateful   Yes        0.23      0.81  0.79   
                     No         0.89      0.81  0.79   
           offensive Yes        0.57      0.76  0.81   
                     No         0.83      0.76  0.81   
           toxic     Yes        0.00      0.82  0.81   
                     No         0.90      0.82  0.81   

                                            Confusion  
size test  Category  Class                             
7B   clean hateful   Yes      [[2223, 474], [54, 79]]  
                     No       [[2223, 474], [54, 79]]  
           offensive Yes    [[1693, 428], [259, 450]]  
                     No     [[1693, 428], [259, 450]]  
           toxic     Yes        [[2323, 506], [0, 1]]  
                     No         [[2323, 506], [0, 1]]

In [43]:
total_metric_table.loc[:, :, 'visual', 0.5]

Support  Support %  Prediction  \
model   size        Category  Class                                   
llama2  13B         hateful   Yes        553   0.195406           0   
                              No        2277   0.804594        2830   
                    offensive Yes        878   0.310247         276   
                              No        1952   0.689753        2554   
                    toxic     Yes        507   0.179152         996   
                              No        2323   0.820848        1834   
        7B          hateful   Yes        553   0.195406           0   
                              No        2277   0.804594        2830   
                    offensive Yes        878   0.310247        1378   
                              No        1952   0.689753        1452   
                    toxic     Yes        507   0.179152         273   
                              No        2323   0.820848        2557   
mistral 7B          hateful   Yes        553   0.195406         378   
                              No        2277   0.804594        2452   
                    offensive Yes        878   0.310247        1851   
                              No        1952   0.689753         979   
                    toxic     Yes        507   0.179152           2   
                              No        2323   0.820848        2828   
        7B-shielded hateful   Yes        553   0.196797         148   
                              No        2257   0.803203        2662   
                    offensive Yes        878   0.312456         839   
                              No        1932   0.687544        1971   
                    toxic     Yes        507   0.180427           2   
                              No        2303   0.819573        2808   

                                     Prediction %  Mean Score  Med Score  \
model   size        Category  Class                                        
llama2  13B         hateful   Yes        0.000000    0.363746        0.2   
                              No         1.000000    0.363746        0.2   
                    offensive Yes        0.097527    0.340989        0.4   
                              No         0.902473    0.340989        0.4   
                    toxic     Yes        0.351943    0.556572        0.6   
                              No         0.648057    0.556572        0.6   
        7B          hateful   Yes        0.000000    0.323428        0.2   
                              No         1.000000    0.323428        0.2   
                    offensive Yes        0.486926    0.359293        0.2   
                              No         0.513074    0.359293        0.2   
                    toxic     Yes        0.096466    0.422792        0.5   
                              No         0.903534    0.422792        0.5   
mistral 7B          hateful   Yes        0.133569    0.576608        0.6   
                              No         0.866431    0.576608        0.6   
                    offensive Yes        0.654064    0.738445        0.8   
                              No         0.345936    0.738445        0.8   
                    toxic     Yes        0.000707    0.760883        0.8   
                              No         0.999293    0.760883        0.8   
        7B-shielded hateful   Yes        0.052669    0.418754        0.4   
                              No         0.947331    0.418754        0.4   
                    offensive Yes        0.298577    0.507295        0.6   
                              No         0.701423    0.507295        0.6   
                    toxic     Yes        0.000712    0.561139        0.7   
                              No         0.999288    0.561139        0.7   

                                      ∆ Score  Precision  Recall  F1-Score  \
model   size        Category  Class                                          
llama2  13B         hateful   Yes    0.120106       0.00 

In [19]:
# Uncomment to Update
# total_metric_table.to_csv('analysis/outputs/hot-llm-total-metrics.csv')

# Cache

In [21]:
total_metric_table = pd.read_csv('analysis/outputs/hot-llm-total-metrics.csv').set_index(['model', 'size', 'test', 'ratio', 'Category', 'Class'])
total_metric_table

Support  Support %  \
model   size        test   ratio Category  Class                       
llama2  13B         clean  0.0   hateful   Yes        553   0.195406   
                                           No        2277   0.804594   
                                 offensive Yes        878   0.310247   
                                           No        1952   0.689753   
                                 toxic     Yes        507   0.179152   
...                                                   ...        ...   
mistral 7B-shielded visual 0.5   hateful   No        2257   0.803203   
                                 offensive Yes        878   0.312456   
                                           No        1932   0.687544   
                                 toxic     Yes        507   0.180427   
                                           No        2303   0.819573   

                                                  Prediction  Prediction %  \
model   size        test   ratio Category  Class                             
llama2  13B         clean  0.0   hateful   Yes             0      0.000000   
                                           No           2830      1.000000   
                                 offensive Yes           375      0.132509   
                                           No           2455      0.867491   
                                 toxic     Yes           434      0.153357   
...                                                      ...           ...   
mistral 7B-shielded visual 0.5   hateful   No           2662      0.947331   
                                 offensive Yes           839      0.298577   
                                           No           1971      0.701423   
                                 toxic     Yes             2      0.000712   
                                           No           2808      0.999288   

                                                  Mean Score  Med Score  \
model   size        test   ratio Category  Class                          
llama2  13B         clean  0.0   hateful   Yes      0.313110        0.2   
                                           No       0.313110        0.2   
                                 offensive Yes      0.337385        0.4   
                                           No       0.337385        0.4   
                                 toxic     Yes      0.419187        0.4   
...                                                      ...        ...   
mistral 7B-shielded visual 0.5   hateful   No       0.418754        0.4   
                                 offensive Yes      0.507295        0.6   
                                           No       0.507295        0.6   
                                 toxic     Yes      0.561139        0.7   
                                           No       0.561139        0.7   

                                                   ∆ Score  Precision  Recall  \
model   size        test   ratio Category  Class                                
llama2  13B         clean  0.0   hateful   Yes         NaN       0.00    0.00   
                                           No          NaN       0.80    1.00   
                                 offensive Yes         NaN       0.79    0.34   
                                           No          NaN       0.76    0.96   
                                 toxic     Yes         NaN       0.63    0.54   
...                                                    ...        ...     ...   
mistral 7B-shielded visual 0.5   hateful   No     0.141922       0.83    0.97   
                                 offensive Yes    0.135231       0.56    0.53   
                                           No     0.135231       0.79    0.81   
                                 toxic     Yes    0.128754       1.00    0.00   
                                           No     0.128754       0.82    1.00   

                                                  F1-Score  Accuracy

In [22]:
total_metric_table.loc[:,:,'clean', :].groupby(['model', 'size']).mean()['AUC']

/tmp/ipykernel_438500/3437096077.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  total_metric_table.loc[:,:,'clean', :].groupby(['model', 'size']).mean()['AUC']


model    size
llama2   13B     0.763333
         7B      0.766667
mistral  7B      0.803333
Name: AUC, dtype: float64

In [23]:
total_metric_table

Support  Support %  \
model   size        test   ratio Category  Class                       
llama2  13B         clean  0.0   hateful   Yes        553   0.195406   
                                           No        2277   0.804594   
                                 offensive Yes        878   0.310247   
                                           No        1952   0.689753   
                                 toxic     Yes        507   0.179152   
...                                                   ...        ...   
mistral 7B-shielded visual 0.5   hateful   No        2257   0.803203   
                                 offensive Yes        878   0.312456   
                                           No        1932   0.687544   
                                 toxic     Yes        507   0.180427   
                                           No        2303   0.819573   

                                                  Prediction  Prediction %  \
model   size        test   ratio Category  Class                             
llama2  13B         clean  0.0   hateful   Yes             0      0.000000   
                                           No           2830      1.000000   
                                 offensive Yes           375      0.132509   
                                           No           2455      0.867491   
                                 toxic     Yes           434      0.153357   
...                                                      ...           ...   
mistral 7B-shielded visual 0.5   hateful   No           2662      0.947331   
                                 offensive Yes           839      0.298577   
                                           No           1971      0.701423   
                                 toxic     Yes             2      0.000712   
                                           No           2808      0.999288   

                                                  Mean Score  Med Score  \
model   size        test   ratio Category  Class                          
llama2  13B         clean  0.0   hateful   Yes      0.313110        0.2   
                                           No       0.313110        0.2   
                                 offensive Yes      0.337385        0.4   
                                           No       0.337385        0.4   
                                 toxic     Yes      0.419187        0.4   
...                                                      ...        ...   
mistral 7B-shielded visual 0.5   hateful   No       0.418754        0.4   
                                 offensive Yes      0.507295        0.6   
                                           No       0.507295        0.6   
                                 toxic     Yes      0.561139        0.7   
                                           No       0.561139        0.7   

                                                   ∆ Score  Precision  Recall  \
model   size        test   ratio Category  Class                                
llama2  13B         clean  0.0   hateful   Yes         NaN       0.00    0.00   
                                           No          NaN       0.80    1.00   
                                 offensive Yes         NaN       0.79    0.34   
                                           No          NaN       0.76    0.96   
                                 toxic     Yes         NaN       0.63    0.54   
...                                                    ...        ...     ...   
mistral 7B-shielded visual 0.5   hateful   No     0.141922       0.83    0.97   
                                 offensive Yes    0.135231       0.56    0.53   
                                           No     0.135231       0.79    0.81   
                                 toxic     Yes    0.128754       1.00    0.00   
                                           No     0.128754       0.82    1.00   

                                                  F1-Score  Accuracy

In [24]:
auc_overall = total_metric_table[total_metric_table.index.get_level_values('test') != 'clean'].pivot_table(['AUC'], index=['model', 'size', 'test'], columns=['ratio']).round(2)

In [25]:
total_metric_table[total_metric_table.index.get_level_values('test') == 'visual'].pivot_table(['AUC'], index=['model', 'size', 'test'], columns=['ratio']).round(2)

AUC            
ratio                      0.125 0.250 0.500
model   size        test                    
llama2  13B         visual  0.73  0.72  0.68
        7B          visual  0.75  0.74  0.73
mistral 7B          visual  0.73  0.69  0.65
        7B-shielded visual   NaN   NaN  0.77

In [26]:
auc_clean = total_metric_table[total_metric_table.index.get_level_values('test') == 'clean'].pivot_table(['AUC'], index=['model', 'size', 'test'], columns=['ratio']).round(2)

auc_clean.droplevel('test').merge(auc_overall, left_index=True, right_index=True)

AUC                  
ratio                 0.000 0.125 0.250 0.500
model   size test                            
llama2  13B  phonetic  0.76  0.74  0.74  0.71
             typo      0.76  0.73  0.72  0.69
             visual    0.76  0.73  0.72  0.68
        7B   phonetic  0.77  0.76  0.74  0.73
             typo      0.77  0.75  0.73  0.71
             visual    0.77  0.75  0.74  0.73
mistral 7B   phonetic  0.80  0.75  0.74  0.70
             typo      0.80  0.74  0.71  0.67
             visual    0.80  0.73  0.69  0.65

In [ ]:
for line in auc_clean.droplevel('test').merge(auc_overall, left_index=True, right_index=True).style.to_latex().split('\n'):
    print(line)

In [28]:
auc_clean = total_metric_table[total_metric_table.index.get_level_values('test') == 'clean'].pivot_table(['AUC'], index=['model', 'size', 'test'], columns=['ratio']).round(2)

auc_clean.droplevel('test').merge(auc_overall, left_index=True, right_index=True)

AUC                  
ratio                 0.000 0.125 0.250 0.500
model   size test                            
llama2  13B  phonetic  0.76  0.74  0.74  0.71
             typo      0.76  0.73  0.72  0.69
             visual    0.76  0.73  0.72  0.68
        7B   phonetic  0.77  0.76  0.74  0.73
             typo      0.77  0.75  0.73  0.71
             visual    0.77  0.75  0.74  0.73
mistral 7B   phonetic  0.80  0.75  0.74  0.70
             typo      0.80  0.74  0.71  0.67
             visual    0.80  0.73  0.69  0.65

In [29]:
total_metric_table[total_metric_table.index.get_level_values('test') != 'clean'].pivot_table(['Mean Score'], index=['model', 'size', 'test'], columns=['ratio']).round(2)

Mean Score            
ratio                             0.125 0.250 0.500
model   size        test                           
llama2  13B         phonetic       0.37  0.39  0.41
                    typo           0.38  0.40  0.43
                    visual         0.39  0.40  0.42
        7B          phonetic       0.36  0.37  0.40
                    typo           0.36  0.37  0.39
                    visual         0.36  0.36  0.37
mistral 7B          phonetic       0.52  0.57  0.63
                    typo           0.56  0.61  0.66
                    visual         0.60  0.65  0.69
        7B-shielded visual          NaN   NaN  0.50

In [30]:
total_metric_table[total_metric_table.index.get_level_values('test') == 'clean'].pivot_table(['Mean Score'], index=['model', 'size', 'test'], columns=['ratio']).round(2)

Mean Score
ratio                     0.0
model   size test            
llama2  13B  clean       0.36
        7B   clean       0.33
mistral 7B   clean       0.44

In [31]:
new_df = pd.DataFrame()
clean_table = total_metric_table[total_metric_table.index.get_level_values('test') == 'clean'].pivot_table(['Mean Score'], index=['model', 'size', 'test'], columns=['ratio'])

for model, size in ([('mistral', '7B'), ('llama2', '13B'), ('llama2', '7B')]):
    test_table = total_metric_table[(total_metric_table.index.get_level_values('test') != 'clean') & (total_metric_table.index.get_level_values('model') == model) &  (total_metric_table.index.get_level_values('size') == size)].pivot_table(['Mean Score'], index=['model', 'size', 'test'], columns=['ratio'])
    for _, ratio in test_table.columns:
        test_table[('Delta', ratio)] = test_table[('Mean Score', ratio)] - clean_table.loc[model, size, 'clean'][('Mean Score', 0.0)]
    new_df = pd.concat([new_df, test_table])
new_df.reset_index().merge(clean_table.reset_index(), on=('model', 'size'), suffixes=('', '__')).drop('test__', axis=1).set_index(['model', 'size', 'test'])[[('Mean Score', 0.0)] + [('Delta', ratio)  for ratio in [0.125, .25, .5]]].round(3)

/tmp/ipykernel_438500/3506052388.py:9: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  new_df.reset_index().merge(clean_table.reset_index(), on=('model', 'size'), suffixes=('', '__')).drop('test__', axis=1).set_index(['model', 'size', 'test'])[[('Mean Score', 0.0)] + [('Delta', ratio)  for ratio in [0.125, .25, .5]]].round(3)
/tmp/ipykernel_438500/3506052388.py:9: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  new_df.reset_index().merge(clean_table.reset_index(), on=('model', 'size'), suffixes=('', '__')).drop('test__', axis=1).set_index(['model', 'size', 'test'])[[('Mean Score', 0.0)] + [('Delta', ratio)  for ratio in [0.125, .25, .5]]].round(3)


Mean Score  Delta              
ratio                        0.0  0.125   0.25    0.5
model   size test                                    
mistral 7B   phonetic      0.438  0.086  0.132  0.194
             typo          0.438  0.117  0.168  0.225
             visual        0.438  0.164  0.214  0.254
llama2  13B  phonetic      0.357  0.017  0.033  0.056
             typo          0.357  0.027  0.048  0.077
             visual        0.357  0.032  0.047  0.064
        7B   phonetic      0.332  0.026  0.042  0.069
             typo          0.332  0.028  0.036  0.054
             visual        0.332  0.026  0.033  0.037

In [43]:
total_metric_table.loc['llama2'].reset_index().groupby(['size', 'test', 'Category', 'ratio']).count()[['Class']]

Class
size test     Category  ratio       
13B  clean    hateful   0.000      2
              offensive 0.000      2
              toxic     0.000      2
     phonetic hateful   0.125      2
                        0.250      2
                        0.500      2
              offensive 0.125      2
                        0.250      2
                        0.500      2
              toxic     0.125      2
                        0.250      2
                        0.500      2
     typo     hateful   0.125      2
                        0.250      2
                        0.500      2
              offensive 0.125      2
                        0.250      2
                        0.500      2
              toxic     0.125      2
                        0.250      2
                        0.500      2
     visual   hateful   0.125      2
                        0.250      2
                        0.500      2
              offensive 0.125      2
                        0.250      2
                        0.500      2
              toxic     0.125      2
                        0.250      2
                        0.500      2
7B   clean    hateful   0.000      2
              offensive 0.000      2
              toxic     0.000      2
     phonetic hateful   0.125      2
                        0.250      2
                        0.500      2
              offensive 0.125      2
                        0.250      2
                        0.500      2
              toxic     0.125      2
                        0.250      2
                        0.500      2
     typo     hateful   0.125      2
                        0.250      2
                        0.500      2
              offensive 0.125      2
                        0.250      2
                        0.500      2
              toxic     0.125      2
                        0.250      2
                        0.500      2
     visual   hateful   0.125      2
                        0.250      2
                        0.500      2
              offensive 0.125      2
                        0.250      2
                        0.500      2
              toxic     0.125      2
                        0.250      2
                        0.500      2

In [33]:
total_metric_table.loc['llama2', :, 'clean'][['AUC']]

AUC
size ratio Category  Class      
13B  0.0   hateful   Yes    0.70
                     No     0.70
           offensive Yes    0.73
                     No     0.73
           toxic     Yes    0.86
                     No     0.86
7B   0.0   hateful   Yes    0.69
                     No     0.69
           offensive Yes    0.78
                     No     0.78
           toxic     Yes    0.83
                     No     0.83

In [34]:
display(total_metric_table.loc['mistral', :, 'clean', 0.0][['AUC']])
display(total_metric_table.loc['mistral', :, 'visual', 0.5][['AUC']])

AUC
size Category  Class      
7B   hateful   Yes    0.79
               No     0.79
     offensive Yes    0.81
               No     0.81
     toxic     Yes    0.81
               No     0.81

AUC
size        Category  Class      
7B          hateful   Yes    0.68
                      No     0.68
            offensive Yes    0.61
                      No     0.61
            toxic     Yes    0.66
                      No     0.66
7B-shielded hateful   Yes    0.77
                      No     0.77
            offensive Yes    0.77
                      No     0.77
            toxic     Yes    0.77
                      No     0.77

# Statistical Difference Tests

In [35]:
from scipy.stats import ttest_rel

# Example data: Replace these with your actual datasets
rows = []
for (model, size) in [('mistral', '7B'), ('llama2', '7B'), ('llama2', '13B')]:
    for ratio in labeled_df.index.get_level_values('ratio').unique():
        if ratio == 0.0:
            continue
        for attack in ['phonetic', 'typo']:
            for clazz in ['hateful', 'offensive', 'toxic']:
                try:
                    clean_score = labeled_df.loc[model, size, 'clean', 0.0][f'{clazz}_score']
                    baseline_score = labeled_df.loc[model, size, attack, ratio][f'{clazz}_score']

                    score_base = (baseline_score - clean_score).abs()

                    visual_score = labeled_df.loc[model, size, 'visual', ratio][f'{clazz}_score']

                    score_visual = (visual_score - clean_score).abs()

                except:
                    continue

                # # Perform the paired t-test
                stat, p = ttest_rel(score_base, score_visual, alternative='less')
                delta = (score_base - score_visual).mean()

                sig_0_5 = p < 0.05
                sig_bon_ratio = p < (0.05/18)
                sig_bon = p < (0.05/54)
                
                rows.append((model, size, ratio, clazz, attack, stat, p, delta,  sig_0_5, sig_bon_ratio, sig_bon))

sig_tests =pd.DataFrame(rows, columns = ['model', 'size', 'ratio', 'clazz', 'baseline', 't-stat', 'p-value', 'mean', 'sig@.05', 'sig_bon_ratio', 'sig_bon_all'])
display(sig_tests.groupby(['model', 'size', 'ratio', 'sig_bon_ratio']).count()[['clazz']])

/tmp/ipykernel_438500/3108520126.py:12: PerformanceWarning: indexing past lexsort depth may impact performance.
  clean_score = labeled_df.loc[model, size, 'clean', 0.0][f'{clazz}_score']
/tmp/ipykernel_438500/3108520126.py:13: PerformanceWarning: indexing past lexsort depth may impact performance.
  baseline_score = labeled_df.loc[model, size, attack, ratio][f'{clazz}_score']
/tmp/ipykernel_438500/3108520126.py:17: PerformanceWarning: indexing past lexsort depth may impact performance.
  visual_score = labeled_df.loc[model, size, 'visual', ratio][f'{clazz}_score']


clazz
model   size ratio sig_bon_ratio       
llama2  13B  0.125 False              1
                   True               5
             0.250 False              3
                   True               3
             0.500 False              4
                   True               2
        7B   0.125 False              4
                   True               2
             0.250 False              6
             0.500 False              6
mistral 7B   0.125 True               6
             0.250 True               6
             0.500 True               6

In [36]:
(0.05/18), (0.05/54)

(0.002777777777777778, 0.000925925925925926)

In [37]:
sig_tests.set_index(['model', 'size', 'ratio', 'baseline']).loc['llama2', '13B'][['clazz', 'sig_bon_ratio']]

/tmp/ipykernel_438500/2419233559.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  sig_tests.set_index(['model', 'size', 'ratio', 'baseline']).loc['llama2', '13B'][['clazz', 'sig_bon_ratio']]


clazz  sig_bon_ratio
ratio baseline                          
0.500 phonetic    hateful          False
      phonetic  offensive           True
      phonetic      toxic           True
      typo        hateful          False
      typo      offensive          False
      typo          toxic          False
0.125 phonetic    hateful           True
      phonetic  offensive           True
      phonetic      toxic           True
      typo        hateful           True
      typo      offensive          False
      typo          toxic           True
0.250 phonetic    hateful           True
      phonetic  offensive          False
      phonetic      toxic           True
      typo        hateful          False
      typo      offensive          False
      typo          toxic           True

In [38]:
sig_tests.set_index(['model', 'size', 'ratio', 'baseline']).loc['llama2', '13B'][['clazz', 'sig_bon_ratio']]

/tmp/ipykernel_438500/2419233559.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  sig_tests.set_index(['model', 'size', 'ratio', 'baseline']).loc['llama2', '13B'][['clazz', 'sig_bon_ratio']]


clazz  sig_bon_ratio
ratio baseline                          
0.500 phonetic    hateful          False
      phonetic  offensive           True
      phonetic      toxic           True
      typo        hateful          False
      typo      offensive          False
      typo          toxic          False
0.125 phonetic    hateful           True
      phonetic  offensive           True
      phonetic      toxic           True
      typo        hateful           True
      typo      offensive          False
      typo          toxic           True
0.250 phonetic    hateful           True
      phonetic  offensive          False
      phonetic      toxic           True
      typo        hateful          False
      typo      offensive          False
      typo          toxic           True

In [39]:
sig_tests.groupby(['model', 'size', 'sig_bon_ratio']).count()

ratio  clazz  baseline  t-stat  p-value  mean  \
model   size sig_bon_ratio                                                  
llama2  13B  False              8      8         8       8        8     8   
             True              10     10        10      10       10    10   
        7B   False             16     16        16      16       16    16   
             True               2      2         2       2        2     2   
mistral 7B   True              18     18        18      18       18    18   

                            sig@.05  sig_bon_all  
model   size sig_bon_ratio                        
llama2  13B  False                8            8  
             True                10           10  
        7B   False               16           16  
             True                 2            2  
mistral 7B   True                18           18

In [40]:
import math
round_to_3 = lambda x: x if x == 0 or np.isnan(x) else round(x, -int(math.floor(math.log10(abs(x)))) + (4 - 1))

sig_tests_for_print = sig_tests.set_index(['model', 'size', 'ratio', 'baseline', 'clazz']).rename({'t-stat':'T-stat', 'p-value': 'P-value', 'sig@.05': 'Sig. at 0.05', 'mean': 'Diff-in-Diff', 'sig_bon_ratio': 'Sig. BC on Model', 'sig_bon_all':'Sig. BC on All'}, axis=1)

cols = ['Diff-in-Diff', 'T-stat', 'P-value']

sig_tests
for col in cols:
    sig_tests_for_print[col] = sig_tests_for_print[col].apply(round_to_3)

print_cols = cols + ['Sig. at 0.05', 'Sig. BC on Model', 'Sig. BC on All']
sig_tests_for_print[print_cols]

Diff-in-Diff    T-stat       P-value  \
model   size ratio baseline clazz                                             
mistral 7B   0.500 phonetic hateful       -0.047880 -11.04000  4.727000e-28   
                            offensive     -0.071020 -15.77000  4.726000e-54   
                            toxic         -0.054310 -12.93000  1.679000e-37   
                   typo     hateful       -0.023850  -5.67900  7.476000e-09   
                            offensive     -0.021410  -5.14900  1.396000e-07   
                            toxic         -0.021020  -5.43900  2.904000e-08   
             0.125 phonetic hateful       -0.048660 -12.01000  9.354000e-33   
                            offensive     -0.088060 -17.96000  1.030000e-68   
                            toxic         -0.059890 -13.21000  5.567000e-39   
                   typo     hateful       -0.028550  -7.17000  4.769000e-13   
                            offensive     -0.050350 -10.66000  2.325000e-26   
                            toxic         -0.034490  -7.63800  1.502000e-14   
             0.250 phonetic hateful       -0.047560 -11.24000  5.011000e-29   
                            offensive     -0.085650 -17.63000  2.187000e-66   
                            toxic         -0.069650 -15.03000  1.793000e-49   
                   typo     hateful       -0.027810  -6.70400  1.215000e-11   
                            offensive     -0.037210  -8.25400  1.160000e-16   
                            toxic         -0.039290  -9.20000  3.367000e-20   
llama2  7B   0.500 phonetic hateful        0.011980   3.37600  9.996000e-01   
                            offensive      0.003852   1.27500  8.989000e-01   
                            toxic          0.004912   1.44300  9.255000e-01   
                   typo     hateful        0.010990   3.23000  9.994000e-01   
                            offensive      0.006113   2.12400  9.831000e-01   
                            toxic          0.011340   3.47500  9.997000e-01   
             0.125 phonetic hateful       -0.007809  -2.71400  3.343000e-03   
                            offensive     -0.007986  -3.27100  5.422000e-04   
                            toxic         -0.007032  -2.50800  6.104000e-03   
                   typo     hateful       -0.007597  -2.70400  3.448000e-03   
                            offensive     -0.007208  -3.03500  1.215000e-03   
                            toxic         -0.002191  -0.79900  2.122000e-01   
             0.250 phonetic hateful        0.000177   0.05522  5.220000e-01   
                            offensive     -0.004629  -1.70500  4.418000e-02   
                            toxic         -0.007032  -2.22400  1.310000e-02   
                   typo     hateful        0.001731   0.54860  7.083000e-01   
                            offensive      0.003251   1.21500  8.878000e-01   
                            toxic         -0.004770  -1.58200  5.692000e-02   
        13B  0.500 phonetic hateful       -0.002014  -0.55340  2.900000e-01   
                            offensive     -0.008198  -3.47300  2.613000e-04   
                            toxic         -0.050740 -13.64000  2.381000e-41   
                   typo     hateful        0.009470   2.64400  9.959000e-01   
                            offensive      0.002792   1.17000  8.790000e-01   
                            toxic         -0.000141  -0.03693  4.853000e-01   
             0.125 phonetic hateful       -0.019290  -6.39800  9.192000e-11   
                            offensive     -0.008057  -3.82600  6.648000e-05   
                            toxic         -0.028480  -9.72400  2.632000e-22   
                   typo     hateful       -0.011380  -3.77200  8.268000e-05   
                            offensive      0.001166   0.55000  7.088000e-01   
                            toxic         -0.008410  -2.78200  2.716000e-03   
             0.250 phonetic hateful       -0.016400  -5.07700  2.042000e-07   
          

In [ ]:
sig_tests_for_print

for line in sig_tests_for_print[print_cols].to_latex().split('\n'):
    print(line)